In [11]:
import mne
import mne_bids
import numpy as np
import pywt
from scipy.stats import entropy
import antropy as ant

In [2]:
def _phi(U, N, m, r):
    z = N - m + 1
    x = np.array([U[i:i+m] for i in range(int(z))])
    X = np.repeat(x[:, np.newaxis], 1, axis=2)
    C = np.sum(np.absolute(x - X).max(axis=2) <= r, axis=0) / z
    return np.log(X).sum() / z

In [3]:
def approx_entropy(sig, win_size, thresh):
    U = np.array(sig)
    N = U.shape[0]
    return abs(_phi(U, N, win_size+1, thresh) - _phi(U, N, win_size, thresh))

In [4]:
def _phi_samp(U, N, m, r):
    z = N - m
    xmi = np.array([U[i:i+m] for i in range(int(z))])
    xmj = np.array([U[i:i+m] for i in range(int(z+1))])
    return np.sum([np.sum(np.abs(xmii - xmj).max(axis=1) <= r) -1 for xmii in xmi])

In [5]:
def sample_entropy(sig, win_size, thresh):
    U = np.array(sig)
    N = U.shape[0]
    return -np.log(_phi_samp(U, N, win_size, 3) / _phi_samp(U, N, win_size+1, 3))

In [12]:
def spectral_entropy(sig, sfreq):
    return ant.spectral_entropy(sig, sfreq)

In [4]:
def total_wavelet_entropy(sig, wname='coif2', level=4):
    y=sig
    n = len(sig)
    energy = np.zeros(level)
    # Calculate energy at each level
    for lev in range(level):
        (y, cD) = pywt.dwt(y, wname)
        energy[lev] = np.sum(np.power(y, 2)) / n
    energy_tot = np.sum(energy)
    p_j = np.array([enr/energy_tot for enr in energy])
    wavelet_entropy = - np.sum(np.dot(p_j, np.log(p_j)))
    return wavelet_entropy   

In [5]:
sig = [1, 2, 3, 4, 5, 1]
we = total_wavelet_entropy(sig)
we

1.1231899524180522

In [13]:
x = np.random.rand(3000)
se = spectral_entropy(x, 100)
se

9.978684580434496